# Environment Setup and Validation

This notebook helps you validate that your environment is properly set up for running LangChain with Ollama.

**Prerequisites:**
- Python 3.8+
- Ollama installed and running locally
- Required packages installed (see requirements.txt)


In [ ]:
# 1. Python Version Check
import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")

# Check minimum version requirement
if sys.version_info >= (3, 8):
    print("✅ Python version meets requirements (3.8+)")
else:
    print("❌ Python version is too old. Please upgrade to 3.8 or newer.")

In [ ]:
# 2. Package Verification
import pkg_resources
import importlib

required_packages = [
    'langchain',
    'langchain_ollama', 
    'numpy',
    'pandas',
    'requests',
    'python-dotenv'
]

print("Checking required packages...")
missing_packages = []

for package in required_packages:
    try:
        # Try to import the package
        importlib.import_module(package.replace('-', '_'))
        # Get version info
        try:
            version = pkg_resources.get_distribution(package).version
            print(f"✅ {package}: {version}")
        except:
            print(f"✅ {package}: installed (version unknown)")
    except ImportError:
        print(f"❌ {package}: not installed")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠️ Missing packages: {', '.join(missing_packages)}")
    print("Install with: pip install -r requirements.txt")
else:
    print("\n✅ All required packages are installed!")

In [ ]:
# 3. Configuration Loading
import os
import sys
sys.path.append('..')  # Add parent directory to path

try:
    from config import config
    print("✅ Configuration loaded successfully")
    print(f"   Ollama URL: {config.ollama_base_url}")
    print(f"   Default model: {config.default_model}")
    print(f"   Temperature: {config.temperature}")
    print(f"   Max tokens: {config.max_tokens}")
except ImportError as e:
    print(f"❌ Error loading configuration: {e}")
    print("Make sure config.py is in the parent directory")

In [ ]:
# 4. Ollama Connection Testing
import requests
from config import config

print("Testing Ollama connection...")
try:
    # Test basic connection
    response = requests.get(f"{config.ollama_base_url}/api/tags", timeout=10)
    
    if response.status_code == 200:
        print("✅ Connection to Ollama successful")
        
        # Show available models
        data = response.json()
        models = [model['name'] for model in data.get('models', [])]
        
        if models:
            print(f"   Available models: {', '.join(models)}")
            
            if config.default_model in [model.split(':')[0] for model in models]:
                print(f"✅ Default model '{config.default_model}' is available")
            else:
                print(f"⚠️ Default model '{config.default_model}' not found")
                print(f"   You may need to run: ollama pull {config.default_model}")
        else:
            print("⚠️ No models found. Pull a model with: ollama pull llama2")
    else:
        print(f"❌ Failed to connect to Ollama, status code: {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("❌ Connection failed - Is Ollama running?")
    print("   Start Ollama with: ollama serve")
except requests.exceptions.Timeout:
    print("❌ Connection timeout - Ollama may be starting up")
except Exception as e:
    print(f"❌ Error connecting to Ollama: {e}")

In [ ]:
# 5. LangChain Integration Test
print("Testing LangChain + Ollama integration...")

try:
    from langchain_ollama import ChatOllama
    
    # Create a simple LLM instance
    llm = ChatOllama(
        model=config.default_model,
        base_url=config.ollama_base_url,
        temperature=0.1  # Low temperature for consistent testing
    )
    
    print("✅ ChatOllama instance created successfully")
    
    # Test a simple prompt
    print("Testing simple prompt...")
    response = llm.invoke("Hello! Please respond with just 'Hello World!'")
    
    print(f"✅ Response received: {response.content[:100]}{'...' if len(response.content) > 100 else ''}")
    
except ImportError as e:
    print(f"❌ LangChain import error: {e}")
except Exception as e:
    print(f"❌ LangChain integration test failed: {e}")
    print("   Make sure Ollama is running and the default model is available")

In [ ]:
# 6. Environment Summary
print("\n" + "="*50)
print("ENVIRONMENT SUMMARY")
print("="*50)

# Python version
python_ok = sys.version_info >= (3, 8)
print(f"Python version: {sys.version.split()[0]} {'✅' if python_ok else '❌'}")

# Package status
packages_ok = len(missing_packages) == 0 if 'missing_packages' in locals() else False
print(f"Required packages: {'✅' if packages_ok else '❌'}")

# Configuration
config_ok = 'config' in locals()
print(f"Configuration: {'✅' if config_ok else '❌'}")

# Ollama connection
ollama_ok = 'response' in locals() and response.status_code == 200
print(f"Ollama connection: {'✅' if ollama_ok else '❌'}")

# LangChain integration
langchain_ok = 'llm' in locals()
print(f"LangChain integration: {'✅' if langchain_ok else '❌'}")

print("\n" + "="*50)
if all([python_ok, packages_ok, config_ok, ollama_ok, langchain_ok]):
    print("🎉 ENVIRONMENT READY! You can proceed with the examples.")
else:
    print("⚠️  SETUP INCOMPLETE. Please resolve the issues above.")
print("="*50)